In [ ]:
import numpy as np
from numpy.linalg import norm, det, inv, LinAlgError
from scipy.spatial import Delaunay, Voronoi, voronoi_plot_2d
from scipy.interpolate import LinearNDInterpolator
from matplotlib import pyplot as plt

from numba import jit, njit

from tqdm import trange

from greenGauss import gaussDiv, gaussGrad, gaussLaplacian

In [ ]:
%matplotlib inline

## 2D divergence test

Evaluate function with known divergence

In [ ]:
x = np.linspace(-1, 1, 21)
y = np.linspace(-1, 1, 21)

xx, yy = np.meshgrid(x, y)

xy = np.transpose([xx.flatten(), yy.flatten()])
R = np.array([
    [np.cos(np.pi/4), np.sin(np.pi/4)],
    [-np.sin(np.pi/4), np.cos(np.pi/4)],
])

xy = xy@R

mesh = Voronoi(xy)

In [ ]:
plt.figure()
voronoi_plot_2d(mesh,)
plt.gca().set_aspect('equal')


In [ ]:
x, y = xy.T

u = np.transpose(
    [x*y, -1/2*y**2]
)

divU = np.zeros(len(u))

In [ ]:
inner_regions = np.array([i for i, region in enumerate(mesh.regions) if -1 not in region])

inner_pts = [-1 not in mesh.regions[region] for region in mesh.point_region]

In [ ]:
div_engine = gaussDiv(mesh, '2d')

In [ ]:
du = div_engine.div(u)

pt_du = du[mesh.point_region]

## 2D gradient test

In [ ]:
p = x*y
grad_p = np.stack([y, x], axis=1)


In [ ]:
grad = gaussGrad(mesh, dim='2d')

In [ ]:
g_p = grad.grad(p)

In [ ]:
err = g_p[mesh.point_region][inner_pts] - grad_p[inner_pts]

_ = plt.hist(err.flatten())

In [ ]:
plt.figure()
plt.tricontourf(x, y, grad_p[:, 0])
plt.colorbar()

plt.figure()
plt.tricontourf(x, y, grad_p[:, 1])
plt.colorbar()


In [ ]:
plt.figure()
plt.tricontourf(x[inner_pts], y[inner_pts], err[:, 0])
plt.colorbar()
plt.figure()
plt.tricontourf(x[inner_pts], y[inner_pts], err[:, 1])
plt.colorbar()

In [ ]:
from matplotlib.colors import LogNorm

plt.tricontourf(
    x[inner_pts],
    y[inner_pts],
    np.abs((pt_du[inner_pts] - divU[inner_pts])),
    levels=16
)
plt.colorbar()
xlim = plt.xlim()
ylim = plt.ylim()
plt.plot(
    mesh.vertices[:, 0],
    mesh.vertices[:, 1],
    ',',
    color='orange'
)

plt.xlim(xlim)
plt.ylim(ylim)

## 2D laplacian test

In [ ]:
q = x**2*y

lap_q = 2*y


In [ ]:
laplacian = gaussLaplacian(mesh, dim='2d')

lq = laplacian.laplace(q)

pt_lq = lq[mesh.point_region]

In [ ]:
l_err = pt_lq[inner_pts] - lap_q[inner_pts]

In [ ]:
plt.figure()
plt.tricontourf(
    x[inner_pts],
    y[inner_pts],
    pt_lq[inner_pts],
    levels=16
)
plt.colorbar()
plt.figure()
plt.tricontourf(
    x[inner_pts],
    y[inner_pts],
    l_err,
    levels=16
)
plt.colorbar()


## 3D divergence test

In [ ]:
x = np.linspace(-1, 1, 21)
y = np.linspace(-1, 1, 21)
z = np.linspace(-1, 1, 21)

xx, yy, zz = np.meshgrid(x, y, z)

xyz = np.transpose([xx.flatten(), yy.flatten(), zz.flatten()])
R = np.array([
    [np.cos(np.pi/4), np.sin(np.pi/4), 0],
    [-np.sin(np.pi/4), np.cos(np.pi/4), 0],
    [0, 0, 1]
])

xyz = xyz@R

mesh = Voronoi(xyz)

In [ ]:
x, y, z = xyz.T

u = np.transpose([
    x*y + x*z,
    -1/2*y**2,
    x**2 + y**2 - 1/2*z**2
])

divU = np.zeros(len(u))

In [ ]:
inner_regions = np.array([
    i 
    for i, region in enumerate(mesh.regions)
    if -1 not in region
])

inner_pts = [
    -1 not in mesh.regions[region]
    for region in mesh.point_region
]

In [ ]:
div3 = gaussDiv(mesh, dim='3d')


In [ ]:
_ = plt.hist(div3.vol, bins=100)

In [ ]:
du = div3.div(u)

pt_du = du[mesh.point_region]

In [ ]:
_ = plt.hist(np.log10(np.abs(pt_du[inner_pts])+1e-15), bins=100, cumulative=True)

## 3D gradient test

In [ ]:
p = x*y*z
grad_p = np.vstack([
    y*z,
    x*z,
    x*y
]).T

In [ ]:
grad = gaussGrad(mesh, '3d')

In [ ]:
gp_est = grad.grad(p)

In [ ]:
gp_pts = gp_est[mesh.point_region]

In [ ]:
err = grad_p[inner_pts] - gp_pts[inner_pts]

In [ ]:
_ = plt.hist(grad_p, bins=100)

In [ ]:
_ = plt.hist(gp_pts[inner_pts], bins=100)

In [ ]:
_ = plt.hist(np.log10(np.abs(err)+1e-6), bins=100)

## 3D laplacian

In [ ]:
q = x**2*y*z
lap_q = 2*y*z

In [ ]:
lap = gaussLaplacian(mesh, '3d')

lp_est = lap(q)

pt_lq = lp_est[mesh.point_region]

In [ ]:
l_err = pt_lq[inner_pts] - lap_q[inner_pts]

In [ ]:
_ = plt.hist(l_err, bins=100)